 # Import the data file, subtract baselines

In [ ]:
from lab_toolbox import lab_toolbox

measurements = lab_toolbox.import_data("./tests/resources/Example_input.txt")

    
''' Remove baseline, default is linear between first and last points. Select more points
    interactively, or remove them by zooming and clicking on them again. The style of 
    interpolation can be changed to: ‘linear’, ‘nearest’, ‘nearest-up’, ‘zero’, 
    ‘slinear’, ‘quadratic’, ‘cubic’, ‘previous’ (scipy.interpolate.interp1d)
    
    If method is not linear, interpolation will behave as linear until minimum of 4 
    points are selected (including start and end)
'''
lab_toolbox.filter_baseline_interactively(measurements, interpolation_method='linear')

# Select start:end of peak

In [ ]:
# Enter the amount of samples taken during the data recording
samples = 2  

# Select integrations of peaks
lab_toolbox.integrate_peaks_interactively(measurements, samples=samples, integration_method='trapz')

# Save data and plots to output folder

In [ ]:
lab_toolbox.output_data(measurements, samples, 'output')